# Polynomial Regression

## Importing the libraries

In [ ]:
from glob import glob
import cv2
from matplotlib import pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate, KFold
from keras.losses import SparseCategoricalCrossentropy, CategoricalCrossentropy
from xgboost import XGBClassifier
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, confusion_matrix, f1_score, precision_score, recall_score, roc_curve, auc
from google.colab import drive
# import glob



from google.colab import drive
drive.mount('/content/drive')

## Importing the dataset

In [ ]:
images_path = {}
images_path["malignant"] = glob("/content/drive/MyDrive/dani/GasHisSDB/120/Abnormal/*.png")
images_path["benign"] = glob("/content/drive/MyDrive/dani/GasHisSDB/120/Normal/*.png")


images_class = {
    'benign' : 0,
    'malignant' : 1
}

import sys

from tensorflow.keras.applications import ResNet50
from keras.models import Model
import keras

pre_trained_models = {}

pre_trained_models = {}


from tensorflow.keras.applications.resnet import ResNet152
import numpy as np

pre_trained_models["ResNet152"] = ResNet152(include_top=False, input_shape=(224, 224, 3), pooling="avg", weights = 'imagenet')
for layer in pre_trained_models["ResNet152"].layers:
    layer.trainable = False


# print(images_path["malignant"])
counter = 0




Y = []
Z = []
pos = 0
for label in images_path:
    counter = 0
    for image_path in images_path[label]:
        # print(counter, ' / ', all)
        counter += 1
        if(1):
          image = cv2.imread(image_path)
          image = cv2.resize(image,(224, 224))
          Z.append(image)
          
          if(counter % 10 == 0):
            temp = np.array(Z)
            

            feature_ResNet152 = pre_trained_models["ResNet152"].predict(temp)
            # print(feature_ResNet152)

            for b in feature_ResNet152:
              c = b.tolist()
              if(pos == 0):
                  X =  [c]
              else:
                X.append(c)


              pos += 1
              # print("shapeX:")
              # o = np.array(X)
              # print(o.shape)


           
            
            # print(X)
            Z = []
          

          Y.append(images_class[label])
          if(counter % 10000  == 0):
            print("counter:")
            print(counter)
            print("label:" + label)
        else:
          continue

print(np.size(X))
print(np.size(Y))







sample_list = X
new_array = np.array(sample_list)

# Displaying the array

file = open("X.txt", "w+")

# Saving the array in a text file
content = str(new_array)
file.write(content)
file.close()



sample_list = Y
new_array = np.array(sample_list)

# Displaying the array

file = open("Y.txt", "w+")

# Saving the array in a text file
content = str(new_array)
file.write(content)
file.close()


234708992/234698864 [==============================] - 3s 0us/step


In [ ]:
x = np.array(X)
y = np.array(Y)


Filename = "res152"


addressX = "/content/drive/MyDrive/dani/120/" + Filename + "_X.txt"
addressY = "/content/drive/MyDrive/dani/120/" + Filename + "_Y.txt"



np.savetxt(addressX, x, fmt='%s')
np.savetxt(addressY, x, fmt='%s')



## Training the Linear Regression model on the whole dataset

In [ ]:
x = np.array(X)
y = np.array(Y)

In [ ]:
from tempfile import TemporaryFile
outfile = TemporaryFile()
np.save(outfile, x)

np.savetxt('X_new.txt', x, fmt='%s')
np.savetxt('Y_new.txt', y, fmt='%s')


In [ ]:
Filename = "res152"


addressX = "/content/drive/MyDrive/dani/" + Filename + "_X.txt"
addressY = "/content/drive/MyDrive/dani/" + Filename + "_Y.txt"



np.savetxt(addressX, x, fmt='%s')
np.savetxt(addressY, x, fmt='%s')



In [ ]:
pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 53 kB/s 


## Visualising the Linear Regression results

In [ ]:
from catboost import CatBoostClassifier
# Initialize data

maxXG_iHolder = 80
maxCAT_iHolder = 61
maxLGBM_iHolder = 92
maxXG = 0
maxCAT = 0
maxLGBM = 0



learnXG_iHolder = 0
learnCAT_iHolder = 0
learnLGBM_iHolder = 0


for i in range(1,11):

  tempL = i / 10

  print("tempL:")
  print(tempL)

  X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=0.3, random_state=maxXG_iHolder)


  result = {}
  Classifier = XGBClassifier(learning_rate=tempL, n_estimators=150, random_state=0, seed=0, gamma=0)
  Classifier.fit(X_train, Y_train)
  Y_pred = Classifier.predict(X_test)
  ac = accuracy_score(Y_test, Y_pred)
  print("Accuracy score : ", ac)

  if(ac > maxXG):
    maxXG = ac
    learnXG_iHolder = i / 10


  X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=0.3, random_state=maxCAT_iHolder)
  

  model = CatBoostClassifier(iterations=1000,learning_rate=tempL,depth=2)
  model.fit(X_train, Y_train,verbose=False)
  Y_pred = model.predict(X_test)
  ac = accuracy_score(Y_test, Y_pred)
  print("Accuracy score : ", ac)

  if(ac > maxCAT):
    maxCAT = ac
    learnCAT_iHolder = i / 10


  X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=0.3, random_state=maxLGBM_iHolder)


  import lightgbm as lgb
  model = lgb.LGBMClassifier(learning_rate=tempL,max_depth=-5,random_state=0)
  model.fit(X_train, Y_train)
  Y_pred = model.predict(X_test)
  ac = accuracy_score(Y_test, Y_pred)
  print("Accuracy score : ", ac)

  if(ac > maxLGBM):
    maxLGBM = ac
    learnLGBM_iHolder = i / 10


  print("maxXG:")
  print(maxXG)

  print("maxCAT:")
  print(maxCAT)

  print("maxLGBM:")
  print(maxLGBM)





  
  print("learnXG_iHolder:")
  print(learnXG_iHolder)

  print("learnCAT_iHolder:")
  print(learnCAT_iHolder)

  print("learnLGBM_iHolder:")
  print(learnLGBM_iHolder)


  
  print("maxXG:")
  print(maxXG)

  print("maxCAT:")
  print(maxCAT)

  print("maxLGBM:")
  print(maxLGBM)


tempL:
0.1
Accuracy score :  0.8902463448828359
Accuracy score :  0.9017624674544362
Accuracy score :  0.9024634488283597
maxXG:
0.8902463448828359
maxCAT:
0.9017624674544362
maxLGBM:
0.9024634488283597
learnXG_iHolder:
0.1
learnCAT_iHolder:
0.1
learnLGBM_iHolder:
0.1
maxXG:
0.8902463448828359
maxCAT:
0.9017624674544362
maxLGBM:
0.9024634488283597
tempL:
0.2
Accuracy score :  0.9014620468656118
Accuracy score :  0.9111756459042659
Accuracy score :  0.9058682155017024
maxXG:
0.9014620468656118
maxCAT:
0.9111756459042659
maxLGBM:
0.9058682155017024
learnXG_iHolder:
0.2
learnCAT_iHolder:
0.2
learnLGBM_iHolder:
0.2
maxXG:
0.9014620468656118
maxCAT:
0.9111756459042659
maxLGBM:
0.9058682155017024
tempL:
0.3
Accuracy score :  0.903264570398558
Accuracy score :  0.9102743841377929
Accuracy score :  0.9058682155017024
maxXG:
0.903264570398558
maxCAT:
0.9111756459042659
maxLGBM:
0.9058682155017024
learnXG_iHolder:
0.3
learnCAT_iHolder:
0.2
learnLGBM_iHolder:
0.2
maxXG:
0.903264570398558
maxCAT:


## Tubeee

In [ ]:

Filename = "res152"


addressX = "/content/drive/MyDrive/dani/" + Filename + "_X.txt"
addressY = "/content/drive/MyDrive/dani/" + "main" + "_Y.txt"





b = np.loadtxt(addressX)
X = b

b = np.loadtxt(addressY)
Y = b

x = np.array(X)
y = np.array(Y)


print(b.shape)
print(x.shape)


(33284,)
(33284, 2048)


In [ ]:
maxXG_iHolder = 80
maxCAT_iHolder = 61
maxLGBM_iHolder = 92

maxXG = 0
learnXG_iHolder = 0.3
learnCAT_iHolder = 0.2
learnLGBM_iHolder = 0.2


X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=0.3, random_state=maxCAT_iHolder)

In [ ]:
pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 1.2 MB/s 


In [ ]:
maxXG = 0
learnXG_iHolder = 0.3
learnCAT_iHolder = 0.2
learnLGBM_iHolder = 0.2


skipper = True

for tempGamma in range(0,51):
  tempGamma = tempGamma / 10
  for min_child_weight in range(1,11):
    for subsample in range(1,11):
      subsample = subsample / 10
      for colsample_bytree in range(1,11):
        colsample_bytree = colsample_bytree / 10
        for max_depth in range(1,11):
          for n_entimatore in range(50,401,50):

            if(skipper):
              result = {}
              Classifier = XGBClassifier(learning_rate=learnXG_iHolder,min_child_weight=min_child_weight,max_depth=max_depth, n_estimators=n_entimatore, subsample=subsample, colsample_bytree=colsample_bytree, gamma=tempGamma)
              Classifier.fit(X_train, Y_train)
              Y_pred = Classifier.predict(X_test)
              ac = accuracy_score(Y_test, Y_pred)
              print("Accuracy score : ", ac)
              print("params:")
              print("gamma:",tempGamma)
              print("subsample:",subsample)
              print("colsample_bytree:",colsample_bytree)
              print("max_depth:",max_depth)
              print("min_child_weight:",min_child_weight)
              print("n_entimatore:",n_entimatore)


              print()
              print("Best_params:")
              print("Best_gamma:",Best_gamma)
              print("Best_subsample:",Best_subsample)
              print("Best_colsample_bytree:",Best_colsample_bytree)
              print("Best_max_depth:",Best_max_depth)
              print("Best_min_child_weight:",Best_min_child_weight)
              print("Best_n_entimatore:",Best_n_entimatore)
              print("Best_ac:",maxXG)

              if(ac > maxXG):
                maxXG = ac
                Best_gamma = tempGamma
                Best_subsample = subsample
                Best_colsample_bytree = colsample_bytree
                Best_max_depth = max_depth
                Best_min_child_weight = min_child_weight
                Best_n_entimatore = n_entimatore


                print("Best_params:")
                print("Best_gamma:",Best_gamma)
                print("Best_subsample:",Best_subsample)
                print("Best_colsample_bytree:",Best_colsample_bytree)
                print("Best_max_depth:",Best_max_depth)
                print("Best_min_child_weight:",Best_min_child_weight)
                print("Best_n_entimatore:",Best_n_entimatore)
                print("Best_ac:",maxXG)








print("Best_params:")
print("Best_gamma:",Best_gamma)
print("Best_subsample:",Best_subsample)
print("Best_colsample_bytree:",Best_colsample_bytree)
print("Best_max_depth:",Best_max_depth)
print("Best_min_child_weight:",Best_min_child_weight)
print("Best_n_entimatore:",Best_n_entimatore)
print("Best_ac:",maxXG)

Streaming output truncated to the last 5000 lines.
min_child_weight: 1
n_entimatore: 50

Best_params:
Best_gamma: 0.0
Best_subsample: 0.1
Best_colsample_bytree: 0.1
Best_max_depth: 1
Best_min_child_weight: 1
Best_n_entimatore: 350
Best_ac: 0.8753254556378931
Accuracy score :  0.8264570398557981
params:
gamma: 0.0
subsample: 0.1
colsample_bytree: 0.1
max_depth: 7
min_child_weight: 1
n_entimatore: 100

Best_params:
Best_gamma: 0.0
Best_subsample: 0.1
Best_colsample_bytree: 0.1
Best_max_depth: 1
Best_min_child_weight: 1
Best_n_entimatore: 350
Best_ac: 0.8753254556378931
Accuracy score :  0.8318646104546364
params:
gamma: 0.0
subsample: 0.1
colsample_bytree: 0.1
max_depth: 7
min_child_weight: 1
n_entimatore: 150

Best_params:
Best_gamma: 0.0
Best_subsample: 0.1
Best_colsample_bytree: 0.1
Best_max_depth: 1
Best_min_child_weight: 1
Best_n_entimatore: 350
Best_ac: 0.8753254556378931
Accuracy score :  0.8280592829961947
params:
gamma: 0.0
subsample: 0.1
colsample_bytree: 0.1
max_depth: 7
min_c

## Gama Test

In [ ]:
maxXG = 0
learnXG_iHolder = 0.3
learnCAT_iHolder = 0.2
learnLGBM_iHolder = 0.2


In [ ]:
maxXG = 0
skipper = False
skipper = True

GammaArr = [0.5,1,1.5,2,5,10]


Best_gamma = 0
Best_subsample = 0
Best_colsample_bytree = 0
Best_max_depth = 0
Best_min_child_weight = 0
Best_n_entimatore = 0



for gamma in GammaArr:
  subsample = 0
  min_child_weight = 0
  colsample_bytree = 0
  max_depth = 0
  n_entimatore = 150

  if(skipper):
    result = {}
    Classifier = XGBClassifier(learning_rate=learnXG_iHolder, gamma=gamma, n_estimators=n_entimatore, random_state=0, seed=0)
    Classifier.fit(X_train, Y_train)
    Y_pred = Classifier.predict(X_test)
    ac = accuracy_score(Y_test, Y_pred)
    print("Accuracy score : ", ac)
    print("params:")
    print("gamma:",gamma)
    print("subsample:",subsample)
    print("colsample_bytree:",colsample_bytree)
    print("max_depth:",max_depth)
    print("min_child_weight:",min_child_weight)
    print("n_entimatore:",n_entimatore)



    print()
    print("Best_params:")
    print("Best_gamma:",Best_gamma)
    print("Best_subsample:",Best_subsample)
    print("Best_colsample_bytree:",Best_colsample_bytree)
    print("Best_max_depth:",Best_max_depth)
    print("Best_min_child_weight:",Best_min_child_weight)
    print("Best_n_entimatore:",Best_n_entimatore)
    print("Best_ac:",maxXG)







    if(ac > maxXG):
      maxXG = ac
      Best_gamma = gamma
      Best_subsample = subsample
      Best_colsample_bytree = colsample_bytree
      Best_max_depth = max_depth
      Best_min_child_weight = min_child_weight
      Best_n_entimatore = n_entimatore


      print("Best_params:")
      print("Best_gamma:",Best_gamma)
      print("Best_subsample:",Best_subsample)
      print("Best_colsample_bytree:",Best_colsample_bytree)
      print("Best_max_depth:",Best_max_depth)
      print("Best_min_child_weight:",Best_min_child_weight)
      print("Best_n_entimatore:",Best_n_entimatore)
      print("Best_ac:",maxXG)








print("Best_params:")
print("Best_gamma:",Best_gamma)
print("Best_subsample:",Best_subsample)
print("Best_colsample_bytree:",Best_colsample_bytree)
print("Best_max_depth:",Best_max_depth)
print("Best_min_child_weight:",Best_min_child_weight)
print("Best_n_entimatore:",Best_n_entimatore)
print("Best_ac:",maxXG)

Accuracy score :  0.903264570398558
params:
gamma: 0.5
subsample: 0
colsample_bytree: 0
max_depth: 0
min_child_weight: 0
n_entimatore: 150

Best_params:
Best_gamma: 0
Best_subsample: 0
Best_colsample_bytree: 0
Best_max_depth: 0
Best_min_child_weight: 0
Best_n_entimatore: 0
Best_ac: 0
Best_params:
Best_gamma: 0.5
Best_subsample: 0
Best_colsample_bytree: 0
Best_max_depth: 0
Best_min_child_weight: 0
Best_n_entimatore: 150
Best_ac: 0.903264570398558
Accuracy score :  0.903264570398558
params:
gamma: 1
subsample: 0
colsample_bytree: 0
max_depth: 0
min_child_weight: 0
n_entimatore: 150

Best_params:
Best_gamma: 0.5
Best_subsample: 0
Best_colsample_bytree: 0
Best_max_depth: 0
Best_min_child_weight: 0
Best_n_entimatore: 150
Best_ac: 0.903264570398558
Accuracy score :  0.903264570398558
params:
gamma: 1.5
subsample: 0
colsample_bytree: 0
max_depth: 0
min_child_weight: 0
n_entimatore: 150

Best_params:
Best_gamma: 0.5
Best_subsample: 0
Best_colsample_bytree: 0
Best_max_depth: 0
Best_min_child_we

In [ ]:
maxXG = 0
skipper = False
skipper = True

gamma = 0.5
GammaArr = [0.5,1,1.5,2,5,10]


Best_gamma = 0.5
Best_subsample = 0
Best_colsample_bytree = 0
Best_max_depth = 0
Best_min_child_weight = 0
Best_n_entimatore = 0



for n_entimatore in range(0,601,50):
  subsample = 0
  min_child_weight = 0
  colsample_bytree = 0
  max_depth = 0

  if(skipper):
    result = {}
    Classifier = XGBClassifier(learning_rate=learnXG_iHolder, gamma=Best_gamma, n_estimators=n_entimatore, random_state=0, seed=0)
    Classifier.fit(X_train, Y_train)
    Y_pred = Classifier.predict(X_test)
    ac = accuracy_score(Y_test, Y_pred)
    print("Accuracy score : ", ac)
    print("params:")
    print("gamma:",gamma)
    print("subsample:",subsample)
    print("colsample_bytree:",colsample_bytree)
    print("max_depth:",max_depth)
    print("min_child_weight:",min_child_weight)
    print("n_entimatore:",n_entimatore)



    print()
    print("Best_params:")
    print("Best_gamma:",Best_gamma)
    print("Best_subsample:",Best_subsample)
    print("Best_colsample_bytree:",Best_colsample_bytree)
    print("Best_max_depth:",Best_max_depth)
    print("Best_min_child_weight:",Best_min_child_weight)
    print("Best_n_entimatore:",Best_n_entimatore)
    print("Best_ac:",maxXG)







    if(ac > maxXG):
      maxXG = ac
      Best_gamma = gamma
      Best_subsample = subsample
      Best_colsample_bytree = colsample_bytree
      Best_max_depth = max_depth
      Best_min_child_weight = min_child_weight
      Best_n_entimatore = n_entimatore


      print("Best_params:")
      print("Best_gamma:",Best_gamma)
      print("Best_subsample:",Best_subsample)
      print("Best_colsample_bytree:",Best_colsample_bytree)
      print("Best_max_depth:",Best_max_depth)
      print("Best_min_child_weight:",Best_min_child_weight)
      print("Best_n_entimatore:",Best_n_entimatore)
      print("Best_ac:",maxXG)








print("Best_params:")
print("Best_gamma:",Best_gamma)
print("Best_subsample:",Best_subsample)
print("Best_colsample_bytree:",Best_colsample_bytree)
print("Best_max_depth:",Best_max_depth)
print("Best_min_child_weight:",Best_min_child_weight)
print("Best_n_entimatore:",Best_n_entimatore)
print("Best_ac:",maxXG)

Accuracy score :  0.6062487482475466
params:
gamma: 0.5
subsample: 0
colsample_bytree: 0
max_depth: 0
min_child_weight: 0
n_entimatore: 0

Best_params:
Best_gamma: 0.5
Best_subsample: 0
Best_colsample_bytree: 0
Best_max_depth: 0
Best_min_child_weight: 0
Best_n_entimatore: 0
Best_ac: 0
Best_params:
Best_gamma: 0.5
Best_subsample: 0
Best_colsample_bytree: 0
Best_max_depth: 0
Best_min_child_weight: 0
Best_n_entimatore: 0
Best_ac: 0.6062487482475466
Accuracy score :  0.8831363909473262
params:
gamma: 0.5
subsample: 0
colsample_bytree: 0
max_depth: 0
min_child_weight: 0
n_entimatore: 50

Best_params:
Best_gamma: 0.5
Best_subsample: 0
Best_colsample_bytree: 0
Best_max_depth: 0
Best_min_child_weight: 0
Best_n_entimatore: 0
Best_ac: 0.6062487482475466
Best_params:
Best_gamma: 0.5
Best_subsample: 0
Best_colsample_bytree: 0
Best_max_depth: 0
Best_min_child_weight: 0
Best_n_entimatore: 50
Best_ac: 0.8831363909473262
Accuracy score :  0.8950530743040256
params:
gamma: 0.5
subsample: 0
colsample_by

In [ ]:
maxXG = 0
skipper = False
skipper = True

Best_gamma = 0.5
tempGamma = 0.5

GammaArr = [0.5,1,1.5,2,5,10]
n_entimatore = 600
Best_n_entimatore = 600


Best_subsample = 0
Best_colsample_bytree = 0
Best_max_depth = 0
Best_min_child_weight = 0


for subsample in range(1,11):
  subsample = subsample / 10
  min_child_weight = 0
  colsample_bytree = 0
  max_depth = 0

  if(skipper):
    result = {}
    Classifier = XGBClassifier(learning_rate=learnXG_iHolder,subsample=subsample, gamma=Best_gamma, n_estimators=n_entimatore, random_state=0, seed=0)
    Classifier.fit(X_train, Y_train)
    Y_pred = Classifier.predict(X_test)
    ac = accuracy_score(Y_test, Y_pred)
    print("Accuracy score : ", ac)
    print("params:")
    print("gamma:",tempGamma)
    print("subsample:",subsample)
    print("colsample_bytree:",colsample_bytree)
    print("max_depth:",max_depth)
    print("min_child_weight:",min_child_weight)
    print("n_entimatore:",n_entimatore)



    print()
    print("Best_params:")
    print("Best_gamma:",Best_gamma)
    print("Best_subsample:",Best_subsample)
    print("Best_colsample_bytree:",Best_colsample_bytree)
    print("Best_max_depth:",Best_max_depth)
    print("Best_min_child_weight:",Best_min_child_weight)
    print("Best_n_entimatore:",Best_n_entimatore)
    print("Best_ac:",maxXG)







    if(ac > maxXG):
      maxXG = ac
      Best_gamma = tempGamma
      Best_subsample = subsample
      Best_colsample_bytree = colsample_bytree
      Best_max_depth = max_depth
      Best_min_child_weight = min_child_weight
      Best_n_entimatore = n_entimatore


      print("Best_params:")
      print("Best_gamma:",Best_gamma)
      print("Best_subsample:",Best_subsample)
      print("Best_colsample_bytree:",Best_colsample_bytree)
      print("Best_max_depth:",Best_max_depth)
      print("Best_min_child_weight:",Best_min_child_weight)
      print("Best_n_entimatore:",Best_n_entimatore)
      print("Best_ac:",maxXG)








print("Best_params:")
print("Best_gamma:",Best_gamma)
print("Best_subsample:",Best_subsample)
print("Best_colsample_bytree:",Best_colsample_bytree)
print("Best_max_depth:",Best_max_depth)
print("Best_min_child_weight:",Best_min_child_weight)
print("Best_n_entimatore:",Best_n_entimatore)
print("Best_ac:",maxXG)

Accuracy score :  0.8459843781293811
params:
gamma: 0.5
subsample: 0.1
colsample_bytree: 0
max_depth: 0
min_child_weight: 0
n_entimatore: 600

Best_params:
Best_gamma: 0.5
Best_subsample: 0
Best_colsample_bytree: 0
Best_max_depth: 0
Best_min_child_weight: 0
Best_n_entimatore: 600
Best_ac: 0
Best_params:
Best_gamma: 0.5
Best_subsample: 0.1
Best_colsample_bytree: 0
Best_max_depth: 0
Best_min_child_weight: 0
Best_n_entimatore: 600
Best_ac: 0.8459843781293811
Accuracy score :  0.8831363909473262
params:
gamma: 0.5
subsample: 0.2
colsample_bytree: 0
max_depth: 0
min_child_weight: 0
n_entimatore: 600

Best_params:
Best_gamma: 0.5
Best_subsample: 0.1
Best_colsample_bytree: 0
Best_max_depth: 0
Best_min_child_weight: 0
Best_n_entimatore: 600
Best_ac: 0.8459843781293811
Best_params:
Best_gamma: 0.5
Best_subsample: 0.2
Best_colsample_bytree: 0
Best_max_depth: 0
Best_min_child_weight: 0
Best_n_entimatore: 600
Best_ac: 0.8831363909473262
Accuracy score :  0.8987582615661927
params:
gamma: 0.5
subsa

## COlsample

In [ ]:
maxXG = 0
skipper = False
skipper = True

GammaArr = [0.5,1,1.5,2,5,10]

Best_gamma = 0.5
tempGamma = 0.5



n_entimatore = 600
Best_n_entimatore = 600


Best_subsample = 1
subsample = 1


Best_colsample_bytree = 0
Best_max_depth = 0
Best_min_child_weight = 0


for colsample_bytree in range(1,11):
  colsample_bytree = colsample_bytree / 10
  min_child_weight = 0
  max_depth = 0

  if(skipper):
    result = {}
    Classifier = XGBClassifier(learning_rate=learnXG_iHolder,colsample_bytree=colsample_bytree,subsample=subsample, gamma=Best_gamma, n_estimators=n_entimatore, random_state=0, seed=0)
    Classifier.fit(X_train, Y_train)
    Y_pred = Classifier.predict(X_test)
    ac = accuracy_score(Y_test, Y_pred)
    print("Accuracy score : ", ac)
    print("params:")
    print("gamma:",tempGamma)
    print("subsample:",subsample)
    print("colsample_bytree:",colsample_bytree)
    print("max_depth:",max_depth)
    print("min_child_weight:",min_child_weight)
    print("n_entimatore:",n_entimatore)



    print()
    print("Best_params:")
    print("Best_gamma:",Best_gamma)
    print("Best_subsample:",Best_subsample)
    print("Best_colsample_bytree:",Best_colsample_bytree)
    print("Best_max_depth:",Best_max_depth)
    print("Best_min_child_weight:",Best_min_child_weight)
    print("Best_n_entimatore:",Best_n_entimatore)
    print("Best_ac:",maxXG)







    if(ac > maxXG):
      maxXG = ac
      Best_gamma = tempGamma
      Best_subsample = subsample
      Best_colsample_bytree = colsample_bytree
      Best_max_depth = max_depth
      Best_min_child_weight = min_child_weight
      Best_n_entimatore = n_entimatore


      print("Best_params:")
      print("Best_gamma:",Best_gamma)
      print("Best_subsample:",Best_subsample)
      print("Best_colsample_bytree:",Best_colsample_bytree)
      print("Best_max_depth:",Best_max_depth)
      print("Best_min_child_weight:",Best_min_child_weight)
      print("Best_n_entimatore:",Best_n_entimatore)
      print("Best_ac:",maxXG)








print("Best_params:")
print("Best_gamma:",Best_gamma)
print("Best_subsample:",Best_subsample)
print("Best_colsample_bytree:",Best_colsample_bytree)
print("Best_max_depth:",Best_max_depth)
print("Best_min_child_weight:",Best_min_child_weight)
print("Best_n_entimatore:",Best_n_entimatore)
print("Best_ac:",maxXG)

Accuracy score :  0.9123773282595634
params:
gamma: 0.5
subsample: 1
colsample_bytree: 0.1
max_depth: 0
min_child_weight: 0
n_entimatore: 600

Best_params:
Best_gamma: 0.5
Best_subsample: 1
Best_colsample_bytree: 0
Best_max_depth: 0
Best_min_child_weight: 0
Best_n_entimatore: 600
Best_ac: 0
Best_params:
Best_gamma: 0.5
Best_subsample: 1
Best_colsample_bytree: 0.1
Best_max_depth: 0
Best_min_child_weight: 0
Best_n_entimatore: 600
Best_ac: 0.9123773282595634
Accuracy score :  0.9155818145403565
params:
gamma: 0.5
subsample: 1
colsample_bytree: 0.2
max_depth: 0
min_child_weight: 0
n_entimatore: 600

Best_params:
Best_gamma: 0.5
Best_subsample: 1
Best_colsample_bytree: 0.1
Best_max_depth: 0
Best_min_child_weight: 0
Best_n_entimatore: 600
Best_ac: 0.9123773282595634
Best_params:
Best_gamma: 0.5
Best_subsample: 1
Best_colsample_bytree: 0.2
Best_max_depth: 0
Best_min_child_weight: 0
Best_n_entimatore: 600
Best_ac: 0.9155818145403565
Accuracy score :  0.9133787302223112
params:
gamma: 0.5
subsa

## DEpth

In [ ]:
maxXG = 0.9122771880632886
skipper = False
skipper = True

GammaArr = [0.5,1,1.5,2,5,10]

Best_gamma = 1
tempGamma = 1



n_entimatore = 550
Best_n_entimatore = 550


Best_subsample = 0.8
subsample = 0.8


Best_colsample_bytree = 1
colsample_bytree = 1

Best_max_depth = 4
Best_min_child_weight = 0


for max_depth in range(10,11):
  min_child_weight = 0

  if(skipper):
    result = {}
    Classifier = XGBClassifier(learning_rate=learnXG_iHolder,max_depth=max_depth,colsample_bytree=Best_colsample_bytree,subsample=subsample, gamma=Best_gamma, n_estimators=n_entimatore, random_state=0, seed=0)
    Classifier.fit(X_train, Y_train)
    Y_pred = Classifier.predict(X_test)
    ac = accuracy_score(Y_test, Y_pred)
    print("Accuracy score : ", ac)
    print("params:")
    print("gamma:",tempGamma)
    print("subsample:",subsample)
    print("colsample_bytree:",colsample_bytree)
    print("max_depth:",max_depth)
    print("min_child_weight:",min_child_weight)
    print("n_entimatore:",n_entimatore)



    print()
    print("Best_params:")
    print("Best_gamma:",Best_gamma)
    print("Best_subsample:",Best_subsample)
    print("Best_colsample_bytree:",Best_colsample_bytree)
    print("Best_max_depth:",Best_max_depth)
    print("Best_min_child_weight:",Best_min_child_weight)
    print("Best_n_entimatore:",Best_n_entimatore)
    print("Best_ac:",maxXG)







    if(ac > maxXG):
      maxXG = ac
      Best_gamma = tempGamma
      Best_subsample = subsample
      Best_colsample_bytree = colsample_bytree
      Best_max_depth = max_depth
      Best_min_child_weight = min_child_weight
      Best_n_entimatore = n_entimatore


      print("Best_params:")
      print("Best_gamma:",Best_gamma)
      print("Best_subsample:",Best_subsample)
      print("Best_colsample_bytree:",Best_colsample_bytree)
      print("Best_max_depth:",Best_max_depth)
      print("Best_min_child_weight:",Best_min_child_weight)
      print("Best_n_entimatore:",Best_n_entimatore)
      print("Best_ac:",maxXG)








print("Best_params:")
print("Best_gamma:",Best_gamma)
print("Best_subsample:",Best_subsample)
print("Best_colsample_bytree:",Best_colsample_bytree)
print("Best_max_depth:",Best_max_depth)
print("Best_min_child_weight:",Best_min_child_weight)
print("Best_n_entimatore:",Best_n_entimatore)
print("Best_ac:",maxXG)

Accuracy score :  0.8995593831363909
params:
gamma: 1
subsample: 0.8
colsample_bytree: 1
max_depth: 10
min_child_weight: 0
n_entimatore: 550

Best_params:
Best_gamma: 1
Best_subsample: 0.8
Best_colsample_bytree: 1
Best_max_depth: 4
Best_min_child_weight: 0
Best_n_entimatore: 550
Best_ac: 0.9122771880632886
Best_params:
Best_gamma: 1
Best_subsample: 0.8
Best_colsample_bytree: 1
Best_max_depth: 4
Best_min_child_weight: 0
Best_n_entimatore: 550
Best_ac: 0.9122771880632886


## Min_child_weight

In [ ]:
maxXG = 0
skipper = False
skipper = True

GammaArr = [0.5,1,1.5,2,5,10]

Best_gamma = 1
tempGamma = 1



n_entimatore = 550
Best_n_entimatore = 550


Best_subsample = 0.8
subsample = 0.8

Best_colsample_bytree = 1
colsample_bytree = 1

# Best_max_depth = 5
Best_min_child_weight = 0


for min_child_weight in range(1,11):

  if(skipper):
    result = {}
    Classifier = XGBClassifier(learning_rate=learnXG_iHolder,min_child_weight=min_child_weight,max_depth=Best_max_depth,colsample_bytree=Best_colsample_bytree,subsample=subsample, gamma=Best_gamma, n_estimators=n_entimatore, random_state=0, seed=0)
    Classifier.fit(X_train, Y_train)
    Y_pred = Classifier.predict(X_test)
    ac = accuracy_score(Y_test, Y_pred)
    print("Accuracy score : ", ac)
    print("params:")
    print("gamma:",tempGamma)
    print("subsample:",subsample)
    print("colsample_bytree:",colsample_bytree)
    print("max_depth:",max_depth)
    print("min_child_weight:",min_child_weight)
    print("n_entimatore:",n_entimatore)



    print()
    print("Best_params:")
    print("Best_gamma:",Best_gamma)
    print("Best_subsample:",Best_subsample)
    print("Best_colsample_bytree:",Best_colsample_bytree)
    print("Best_max_depth:",Best_max_depth)
    print("Best_min_child_weight:",Best_min_child_weight)
    print("Best_n_entimatore:",Best_n_entimatore)
    print("Best_ac:",maxXG)







    if(ac > maxXG):
      maxXG = ac
      Best_gamma = tempGamma
      Best_subsample = subsample
      Best_colsample_bytree = colsample_bytree
      Best_max_depth = max_depth
      Best_min_child_weight = min_child_weight
      Best_n_entimatore = n_entimatore


      print("Best_params:")
      print("Best_gamma:",Best_gamma)
      print("Best_subsample:",Best_subsample)
      print("Best_colsample_bytree:",Best_colsample_bytree)
      print("Best_max_depth:",Best_max_depth)
      print("Best_min_child_weight:",Best_min_child_weight)
      print("Best_n_entimatore:",Best_n_entimatore)
      print("Best_ac:",maxXG)








print("Best_params:")
print("Best_gamma:",Best_gamma)
print("Best_subsample:",Best_subsample)
print("Best_colsample_bytree:",Best_colsample_bytree)
print("Best_max_depth:",Best_max_depth)
print("Best_min_child_weight:",Best_min_child_weight)
print("Best_n_entimatore:",Best_n_entimatore)
print("Best_ac:",maxXG)

Accuracy score :  0.9122771880632886
params:
gamma: 1
subsample: 0.8
colsample_bytree: 1
max_depth: 10
min_child_weight: 1
n_entimatore: 550

Best_params:
Best_gamma: 1
Best_subsample: 0.8
Best_colsample_bytree: 1
Best_max_depth: 4
Best_min_child_weight: 0
Best_n_entimatore: 550
Best_ac: 0
Best_params:
Best_gamma: 1
Best_subsample: 0.8
Best_colsample_bytree: 1
Best_max_depth: 10
Best_min_child_weight: 1
Best_n_entimatore: 550
Best_ac: 0.9122771880632886


## Light

In [ ]:
import lightgbm as lgb

maxXG = 0
Best_numLeaves = 0
skipper = False
skipper = True

Best_min_child_weight = 0


for numLeaves in range(10,110):

  if(skipper):
    model = lgb.LGBMClassifier(learning_rate=learnLGBM_iHolder,num_leaves = numLeaves)
    model.fit(X_train, Y_train)
    Y_pred = model.predict(X_test)
    ac = accuracy_score(Y_test, Y_pred)
    print("Accuracy score : ", ac)

    print("Accuracy score : ", ac)
    print("params:")
    print("numLeaves:",numLeaves)


    print()
    print("Best_params:")
    print("Best_numLeaves:",Best_numLeaves)
    print("Best_ac:",maxXG)






    if(ac > maxXG):
      maxXG = ac
      Best_numLeaves = numLeaves

      print("Best_params:")
      print("Best_numLeaves:",Best_numLeaves)
      print("Best_ac:",maxXG)







print("Best_params:")
print("Best_numLeaves:",Best_numLeaves)
print("Best_ac:",maxXG)










  

Accuracy score :  0.8989585419587423
Accuracy score :  0.8989585419587423
params:
numLeaves: 10

Best_params:
Best_numLeaves: 0
Best_ac: 0
Best_params:
Best_numLeaves: 10
Best_ac: 0.8989585419587423
Accuracy score :  0.8989585419587423
Accuracy score :  0.8989585419587423
params:
numLeaves: 11

Best_params:
Best_numLeaves: 10
Best_ac: 0.8989585419587423
Accuracy score :  0.9031644302022832
Accuracy score :  0.9031644302022832
params:
numLeaves: 12

Best_params:
Best_numLeaves: 10
Best_ac: 0.8989585419587423
Best_params:
Best_numLeaves: 12
Best_ac: 0.9031644302022832
Accuracy score :  0.901862607650711
Accuracy score :  0.901862607650711
params:
numLeaves: 13

Best_params:
Best_numLeaves: 12
Best_ac: 0.9031644302022832
Accuracy score :  0.9035649909873823
Accuracy score :  0.9035649909873823
params:
numLeaves: 14

Best_params:
Best_numLeaves: 12
Best_ac: 0.9031644302022832
Best_params:
Best_numLeaves: 14
Best_ac: 0.9035649909873823
Accuracy score :  0.9045663929501302
Accuracy score :  

## min_child_samples

In [ ]:
import lightgbm as lgb

maxXG = 0
Best_numLeaves = 70
Best_min_child_samples = 0
skipper = False
skipper = True

for min_child_samples in range(0,1250,250):

  if(skipper):
    model = lgb.LGBMClassifier(learning_rate=learnLGBM_iHolder,min_child_samples = min_child_samples,num_leaves = Best_numLeaves)
    model.fit(X_train, Y_train)
    Y_pred = model.predict(X_test)
    ac = accuracy_score(Y_test, Y_pred)
    print("Accuracy score : ", ac)

    print("Accuracy score : ", ac)
    print("params:")
    print("numLeaves:",Best_numLeaves)
    print("min_child_samples:",min_child_samples)


    print()
    print("Best_params:")
    print("Best_numLeaves:",Best_numLeaves)
    print("Best_min_child_samples:",Best_min_child_samples)
    print("Best_ac:",maxXG)






    if(ac > maxXG):
      maxXG = ac
      Best_min_child_samples = min_child_samples

      print("Best_params:")
      print("Best_numLeaves:",Best_numLeaves)
      print("Best_min_child_samples:",Best_min_child_samples)
      print("Best_ac:",maxXG)



print("Best_params:")
print("Best_numLeaves:",Best_numLeaves)
print("Best_min_child_samples:",Best_min_child_samples)
print("Best_ac:",maxXG) 

Accuracy score :  0.9042659723613058
Accuracy score :  0.9042659723613058
params:
numLeaves: 70
min_child_samples: 0

Best_params:
Best_numLeaves: 70
Best_min_child_samples: 0
Best_ac: 0
Best_params:
Best_numLeaves: 70
Best_min_child_samples: 0
Best_ac: 0.9042659723613058
Accuracy score :  0.9084718606048467
Accuracy score :  0.9084718606048467
params:
numLeaves: 70
min_child_samples: 250

Best_params:
Best_numLeaves: 70
Best_min_child_samples: 0
Best_ac: 0.9042659723613058
Best_params:
Best_numLeaves: 70
Best_min_child_samples: 250
Best_ac: 0.9084718606048467
Accuracy score :  0.9063689164830763
Accuracy score :  0.9063689164830763
params:
numLeaves: 70
min_child_samples: 500

Best_params:
Best_numLeaves: 70
Best_min_child_samples: 250
Best_ac: 0.9084718606048467
Accuracy score :  0.9073703184458242
Accuracy score :  0.9073703184458242
params:
numLeaves: 70
min_child_samples: 750

Best_params:
Best_numLeaves: 70
Best_min_child_samples: 250
Best_ac: 0.9084718606048467
Accuracy score : 

## Max_bin

In [ ]:
import lightgbm as lgb

maxXG = 0
Best_numLeaves = 63
Best_min_child_samples = 250
Best_max_bin = 0
skipper = False
skipper = True

for max_bin in range(3,21,3):

  if(skipper):
    model = lgb.LGBMClassifier(learning_rate=learnLGBM_iHolder,max_bin=max_bin,min_child_samples = Best_min_child_samples,num_leaves = Best_numLeaves)
    model.fit(X_train, Y_train)
    Y_pred = model.predict(X_test)
    ac = accuracy_score(Y_test, Y_pred)
    print("Accuracy score : ", ac)

    print("Accuracy score : ", ac)
    print("params:")
    print("numLeaves:",Best_numLeaves)
    print("min_child_samples:",min_child_samples)
    print("max_bin:",max_bin)
    


    print()
    print("Best_params:")
    print("Best_numLeaves:",Best_numLeaves)
    print("Best_min_child_samples:",Best_min_child_samples)
    print("Best_max_bin:",Best_max_bin)    
    print("Best_ac:",maxXG)






    if(ac > maxXG):
      maxXG = ac
      Best_max_bin = max_bin

      print("Best_params:")
      print("Best_numLeaves:",Best_numLeaves)
      print("Best_min_child_samples:",Best_min_child_samples)
      print("Best_max_bin:",Best_max_bin)    
      print("Best_ac:",maxXG)



print("Best_params:")
print("Best_numLeaves:",Best_numLeaves)
print("Best_min_child_samples:",Best_min_child_samples)
print("Best_max_bin:",Best_max_bin)    
print("Best_ac:",maxXG) 

Accuracy score :  0.8917484478269577
Accuracy score :  0.8917484478269577
params:
numLeaves: 63
min_child_samples: 1000
max_bin: 3

Best_params:
Best_numLeaves: 63
Best_min_child_samples: 250
Best_max_bin: 0
Best_ac: 0
Best_params:
Best_numLeaves: 63
Best_min_child_samples: 250
Best_max_bin: 3
Best_ac: 0.8917484478269577
Accuracy score :  0.9042659723613058
Accuracy score :  0.9042659723613058
params:
numLeaves: 63
min_child_samples: 1000
max_bin: 6

Best_params:
Best_numLeaves: 63
Best_min_child_samples: 250
Best_max_bin: 3
Best_ac: 0.8917484478269577
Best_params:
Best_numLeaves: 63
Best_min_child_samples: 250
Best_max_bin: 6
Best_ac: 0.9042659723613058
Accuracy score :  0.9038654115762067
Accuracy score :  0.9038654115762067
params:
numLeaves: 63
min_child_samples: 1000
max_bin: 9

Best_params:
Best_numLeaves: 63
Best_min_child_samples: 250
Best_max_bin: 6
Best_ac: 0.9042659723613058
Accuracy score :  0.9089725615862208
Accuracy score :  0.9089725615862208
params:
numLeaves: 63
min_c

## Best_bagging_fraction

In [ ]:
import lightgbm as lgb

maxXG = 0
Best_numLeaves = 63
Best_min_child_samples = 250
Best_max_bin = 12
Best_bagging_fraction = 0

skipper = False
skipper = True

bagging_fractionArr = [0.6,0.8,1]


for bagging_fraction in bagging_fractionArr:

  if(skipper):
    model = lgb.LGBMClassifier(learning_rate=learnLGBM_iHolder,bagging_fraction=bagging_fraction,max_bin=Best_max_bin,min_child_samples = Best_min_child_samples,num_leaves = Best_numLeaves)
    model.fit(X_train, Y_train)
    Y_pred = model.predict(X_test)
    ac = accuracy_score(Y_test, Y_pred)
    print("Accuracy score : ", ac)

    print("Accuracy score : ", ac)
    print("params:")
    print("numLeaves:",Best_numLeaves)
    print("min_child_samples:",Best_min_child_samples)
    print("bagging_fraction:",bagging_fraction)
    print("max_bin:",Best_max_bin)
    


    print()
    print("Best_params:")
    print("Best_numLeaves:",Best_numLeaves)
    print("Best_min_child_samples:",Best_min_child_samples)
    print("Best_max_bin:",Best_max_bin)    
    print("Best_bagging_fraction:",Best_bagging_fraction)
    print("Best_ac:",maxXG)






    if(ac > maxXG):
      maxXG = ac
      Best_bagging_fraction = bagging_fraction

      print("Best_params:")
      print("Best_numLeaves:",Best_numLeaves)
      print("Best_min_child_samples:",Best_min_child_samples)
      print("Best_max_bin:",Best_max_bin)    
      print("Best_bagging_fraction:",Best_bagging_fraction)
      print("Best_ac:",maxXG)



print("Best_params:")
print("Best_numLeaves:",Best_numLeaves)
print("Best_min_child_samples:",Best_min_child_samples)
print("Best_max_bin:",Best_max_bin)    
print("Best_bagging_fraction:",Best_bagging_fraction)
print("Best_ac:",maxXG) 

Accuracy score :  0.9089725615862208
Accuracy score :  0.9089725615862208
params:
numLeaves: 63
min_child_samples: 250
bagging_fraction: 0.6
max_bin: 12

Best_params:
Best_numLeaves: 63
Best_min_child_samples: 250
Best_max_bin: 12
Best_bagging_fraction: 0
Best_ac: 0
Best_params:
Best_numLeaves: 63
Best_min_child_samples: 250
Best_max_bin: 12
Best_bagging_fraction: 0.6
Best_ac: 0.9089725615862208
Accuracy score :  0.9089725615862208
Accuracy score :  0.9089725615862208
params:
numLeaves: 63
min_child_samples: 250
bagging_fraction: 0.8
max_bin: 12

Best_params:
Best_numLeaves: 63
Best_min_child_samples: 250
Best_max_bin: 12
Best_bagging_fraction: 0.6
Best_ac: 0.9089725615862208
Accuracy score :  0.9089725615862208
Accuracy score :  0.9089725615862208
params:
numLeaves: 63
min_child_samples: 250
bagging_fraction: 1
max_bin: 12

Best_params:
Best_numLeaves: 63
Best_min_child_samples: 250
Best_max_bin: 12
Best_bagging_fraction: 0.6
Best_ac: 0.9089725615862208


LightGBMError: ignored

## Max_depth

In [ ]:
import lightgbm as lgb

maxXG = 0
Best_numLeaves = 63
Best_min_child_samples = 250
Best_max_bin = 12
Best_max_depth = 0

skipper = False
skipper = True


for max_depth in range(2,17):

  if(skipper):
    model = lgb.LGBMClassifier(learning_rate=learnLGBM_iHolder,max_depth=max_depth,max_bin=Best_max_bin,min_child_samples = Best_min_child_samples,num_leaves = Best_numLeaves)
    model.fit(X_train, Y_train)
    Y_pred = model.predict(X_test)
    ac = accuracy_score(Y_test, Y_pred)
    print("Accuracy score : ", ac)

    print("Accuracy score : ", ac)
    print("params:")
    print("numLeaves:",Best_numLeaves)
    print("min_child_samples:",Best_min_child_samples)
    print("max_depth:",max_depth)
    print("max_bin:",Best_max_bin)
    


    print()
    print("Best_params:")
    print("Best_numLeaves:",Best_numLeaves)
    print("Best_min_child_samples:",Best_min_child_samples)
    print("Best_max_bin:",Best_max_bin)    
    print("Best_max_depth:",Best_max_depth)
    print("Best_ac:",maxXG)






    if(ac > maxXG):
      maxXG = ac
      Best_max_depth = max_depth

      print("Best_params:")
      print("Best_numLeaves:",Best_numLeaves)
      print("Best_min_child_samples:",Best_min_child_samples)
      print("Best_max_bin:",Best_max_bin)    
      print("Best_max_depth:",Best_max_depth)
      print("Best_ac:",maxXG)



print("Best_params:")
print("Best_numLeaves:",Best_numLeaves)
print("Best_min_child_samples:",Best_min_child_samples)
print("Best_max_bin:",Best_max_bin)    
print("Best_max_depth:",Best_max_depth)
print("Best_ac:",maxXG) 

Accuracy score :  0.8805327458441818
Accuracy score :  0.8805327458441818
params:
numLeaves: 63
min_child_samples: 250
max_depth: 2
max_bin: 12

Best_params:
Best_numLeaves: 63
Best_min_child_samples: 250
Best_max_bin: 12
Best_max_depth: 0
Best_ac: 0
Best_params:
Best_numLeaves: 63
Best_min_child_samples: 250
Best_max_bin: 12
Best_max_depth: 2
Best_ac: 0.8805327458441818
Accuracy score :  0.8929501301822551
Accuracy score :  0.8929501301822551
params:
numLeaves: 63
min_child_samples: 250
max_depth: 3
max_bin: 12

Best_params:
Best_numLeaves: 63
Best_min_child_samples: 250
Best_max_bin: 12
Best_max_depth: 2
Best_ac: 0.8805327458441818
Best_params:
Best_numLeaves: 63
Best_min_child_samples: 250
Best_max_bin: 12
Best_max_depth: 3
Best_ac: 0.8929501301822551
Accuracy score :  0.8976567194071701
Accuracy score :  0.8976567194071701
params:
numLeaves: 63
min_child_samples: 250
max_depth: 4
max_bin: 12

Best_params:
Best_numLeaves: 63
Best_min_child_samples: 250
Best_max_bin: 12
Best_max_dept

## Best_min_child_weight

In [ ]:
import lightgbm as lgb

maxXG = 0
Best_numLeaves = 87
Best_min_child_samples = 500
Best_max_bin = 15
Best_max_depth = 16
Best_min_child_weight = 0

skipper = False
skipper = True

for min_child_weight in range(20,50):

  if(skipper):
    model = lgb.LGBMClassifier(learning_rate=learnLGBM_iHolder,min_child_weight=min_child_weight,max_depth=Best_max_depth,max_bin=Best_max_bin,min_child_samples = Best_min_child_samples,num_leaves = Best_numLeaves)
    model.fit(X_train, Y_train)
    Y_pred = model.predict(X_test)
    ac = accuracy_score(Y_test, Y_pred)
    print("Accuracy score : ", ac)

    print("Accuracy score : ", ac)
    print("params:")
    print("numLeaves:",Best_numLeaves)
    print("min_child_samples:",Best_min_child_samples)
    print("bagging_fraction:",Best_bagging_fraction)
    print("max_bin:",Best_max_bin)
    print("max_depth:",Best_max_depth)    
    print("min_child_weight:",min_child_weight)    


    print()
    print("Best_params:")
    print("Best_numLeaves:",Best_numLeaves)
    print("Best_min_child_samples:",Best_min_child_samples)
    print("Best_max_bin:",Best_max_bin)    
    print("Best_bagging_fraction:",Best_bagging_fraction)
    print("Best_max_depth:",Best_max_depth)
    print("Best_min_child_weight:",Best_min_child_weight)    
    print("Best_ac:",maxXG)






    if(ac > maxXG):
      maxXG = ac
      Best_min_child_weight = min_child_weight

      print("Best_params:")
      print("Best_numLeaves:",Best_numLeaves)
      print("Best_min_child_samples:",Best_min_child_samples)
      print("Best_max_bin:",Best_max_bin)    
      print("Best_bagging_fraction:",Best_bagging_fraction)
      print("Best_max_depth:",Best_max_depth)
      print("Best_min_child_weight:",Best_min_child_weight)    
      print("Best_ac:",maxXG)



print("Best_params:")
print("Best_numLeaves:",Best_numLeaves)
print("Best_min_child_samples:",Best_min_child_samples)
print("Best_max_bin:",Best_max_bin)    
print("Best_bagging_fraction:",Best_bagging_fraction)
print("Best_max_depth:",Best_max_depth)
print("Best_min_child_weight:",Best_min_child_weight)    
print("Best_ac:",maxXG) 

Accuracy score :  0.9059683556979772
Accuracy score :  0.9059683556979772
params:
numLeaves: 87
min_child_samples: 500
bagging_fraction: 1.2
max_bin: 15
max_depth: 16
min_child_weight: 20

Best_params:
Best_numLeaves: 87
Best_min_child_samples: 500
Best_max_bin: 15
Best_bagging_fraction: 1.2
Best_max_depth: 16
Best_min_child_weight: 0
Best_ac: 0
Best_params:
Best_numLeaves: 87
Best_min_child_samples: 500
Best_max_bin: 15
Best_bagging_fraction: 1.2
Best_max_depth: 16
Best_min_child_weight: 20
Best_ac: 0.9059683556979772
Accuracy score :  0.9054676547166033
Accuracy score :  0.9054676547166033
params:
numLeaves: 87
min_child_samples: 500
bagging_fraction: 1.2
max_bin: 15
max_depth: 16
min_child_weight: 21

Best_params:
Best_numLeaves: 87
Best_min_child_samples: 500
Best_max_bin: 15
Best_bagging_fraction: 1.2
Best_max_depth: 16
Best_min_child_weight: 20
Best_ac: 0.9059683556979772
Accuracy score :  0.9068696174644503
Accuracy score :  0.9068696174644503
params:
numLeaves: 87
min_child_sam

## CAT

In [ ]:
from catboost import CatBoostClassifier

maxXG = 0
Best_numLeaves = 0
skipper = False
skipper = True

Best_iterations = 0


for iterations in range(100,1000,100):

  if(skipper):



    model = CatBoostClassifier(iterations=iterations,learning_rate=learnCAT_iHolder,depth=2)
    model.fit(X_train, Y_train,verbose=False)
    Y_pred = model.predict(X_test)
    ac = accuracy_score(Y_test, Y_pred)

    print("Accuracy score : ", ac)

    print("Accuracy score : ", ac)
    print("params:")
    print("iterations:",iterations)


    print()
    print("Best_params:")
    print("Best_iterations:",Best_iterations)
    print("Best_ac:",maxXG)


    if(ac > maxXG):
      maxXG = ac
      Best_iterations = iterations

      print("Best_params:")
      print("Best_iterations:",Best_iterations)
      print("Best_ac:",maxXG)







print("Best_params:")
print("Best_iterations:",Best_iterations)
print("Best_ac:",maxXG)

Accuracy score :  0.8780292409373123
Accuracy score :  0.8780292409373123
params:
iterations: 100

Best_params:
Best_iterations: 0
Best_ac: 0
Best_params:
Best_iterations: 100
Best_ac: 0.8780292409373123
Accuracy score :  0.8905467654716603
Accuracy score :  0.8905467654716603
params:
iterations: 200

Best_params:
Best_iterations: 100
Best_ac: 0.8780292409373123
Best_params:
Best_iterations: 200
Best_ac: 0.8905467654716603
Accuracy score :  0.8977568596034449
Accuracy score :  0.8977568596034449
params:
iterations: 300

Best_params:
Best_iterations: 200
Best_ac: 0.8905467654716603
Best_params:
Best_iterations: 300
Best_ac: 0.8977568596034449
Accuracy score :  0.9010614860805127
Accuracy score :  0.9010614860805127
params:
iterations: 400

Best_params:
Best_iterations: 300
Best_ac: 0.8977568596034449
Best_params:
Best_iterations: 400
Best_ac: 0.9010614860805127
Accuracy score :  0.9024634488283597
Accuracy score :  0.9024634488283597
params:
iterations: 500

Best_params:
Best_iterations

In [ ]:
from catboost import CatBoostClassifier

maxXG = 0
Best_numLeaves = 0
skipper = False
skipper = True

Best_iterations = 1000
Best_l2_leaf_reg = 0

l2_leaf_regArr = [1,3,5,10,100]


for l2_leaf_reg in l2_leaf_regArr:

  if(skipper):

    model = CatBoostClassifier(iterations=Best_iterations,l2_leaf_reg=l2_leaf_reg,learning_rate=learnCAT_iHolder,depth=2)
    model.fit(X_train, Y_train,verbose=False)
    Y_pred = model.predict(X_test)
    ac = accuracy_score(Y_test, Y_pred)

    print("Accuracy score : ", ac)

    print("Accuracy score : ", ac)
    print("params:")
    print("iterations:",iterations)
    print("l2_leaf_reg:",l2_leaf_reg)
    


    print()
    print("Best_params:")
    print("Best_iterations:",Best_iterations)
    print("Best_l2_leaf_reg:",Best_l2_leaf_reg)
    print("Best_ac:",maxXG)


    if(ac > maxXG):
      maxXG = ac
      Best_l2_leaf_reg = l2_leaf_reg

      print("Best_params:")
      print("Best_iterations:",Best_iterations)
      print("Best_l2_leaf_reg:",Best_l2_leaf_reg)
      print("Best_ac:",maxXG)







print("Best_params:")
print("Best_iterations:",Best_iterations)
print("Best_l2_leaf_reg:",Best_l2_leaf_reg)
print("Best_ac:",maxXG)

Accuracy score :  0.9094732625675946
Accuracy score :  0.9094732625675946
params:
iterations: 900
l2_leaf_reg: 1

Best_params:
Best_iterations: 1000
Best_l2_leaf_reg: 0
Best_ac: 0
Best_params:
Best_iterations: 1000
Best_l2_leaf_reg: 1
Best_ac: 0.9094732625675946
Accuracy score :  0.9111756459042659
Accuracy score :  0.9111756459042659
params:
iterations: 900
l2_leaf_reg: 3

Best_params:
Best_iterations: 1000
Best_l2_leaf_reg: 1
Best_ac: 0.9094732625675946
Best_params:
Best_iterations: 1000
Best_l2_leaf_reg: 3
Best_ac: 0.9111756459042659
Accuracy score :  0.9087722811936711
Accuracy score :  0.9087722811936711
params:
iterations: 900
l2_leaf_reg: 5

Best_params:
Best_iterations: 1000
Best_l2_leaf_reg: 3
Best_ac: 0.9111756459042659
Accuracy score :  0.908371720408572
Accuracy score :  0.908371720408572
params:
iterations: 900
l2_leaf_reg: 10

Best_params:
Best_iterations: 1000
Best_l2_leaf_reg: 3
Best_ac: 0.9111756459042659
Accuracy score :  0.9095734027638694
Accuracy score :  0.9095734

## Depth

In [ ]:
from catboost import CatBoostClassifier

maxXG = 0
Best_numLeaves = 0
skipper = False
skipper = True

Best_iterations = 1000
Best_l2_leaf_reg = 3
Best_depth = 0

l2_leaf_regArr = [1,3,5,10,100]


for depth in range(1,11):

  if(skipper):

    model = CatBoostClassifier(iterations=Best_iterations,l2_leaf_reg=Best_l2_leaf_reg,learning_rate=learnCAT_iHolder,depth=depth)
    model.fit(X_train, Y_train,verbose=False)
    Y_pred = model.predict(X_test)
    ac = accuracy_score(Y_test, Y_pred)

    print("Accuracy score : ", ac)

    print("Accuracy score : ", ac)
    print("params:")
    print("iterations:",Best_iterations)
    print("l2_leaf_reg:",Best_l2_leaf_reg)
    print("depth:",depth)
    


    print()
    print("Best_params:")
    print("Best_iterations:",Best_iterations)
    print("Best_l2_leaf_reg:",Best_l2_leaf_reg)
    print("Best_depth:",Best_depth)
    print("Best_ac:",maxXG)


    if(ac > maxXG):
      maxXG = ac
      Best_depth = depth

      print("Best_params:")
      print("Best_iterations:",Best_iterations)
      print("Best_l2_leaf_reg:",Best_l2_leaf_reg)
      print("Best_depth:",Best_depth)
      print("Best_ac:",maxXG)







print("Best_params:")
print("Best_iterations:",Best_iterations)
print("Best_l2_leaf_reg:",Best_l2_leaf_reg)
print("Best_depth:",Best_depth)
print("Best_ac:",maxXG)

Accuracy score :  0.8962547566593231
Accuracy score :  0.8962547566593231
params:
iterations: 1000
l2_leaf_reg: 3
depth: 1

Best_params:
Best_iterations: 1000
Best_l2_leaf_reg: 3
Best_depth: 0
Best_ac: 0
Best_params:
Best_iterations: 1000
Best_l2_leaf_reg: 3
Best_depth: 1
Best_ac: 0.8962547566593231
Accuracy score :  0.9111756459042659
Accuracy score :  0.9111756459042659
params:
iterations: 1000
l2_leaf_reg: 3
depth: 2

Best_params:
Best_iterations: 1000
Best_l2_leaf_reg: 3
Best_depth: 1
Best_ac: 0.8962547566593231
Best_params:
Best_iterations: 1000
Best_l2_leaf_reg: 3
Best_depth: 2
Best_ac: 0.9111756459042659
Accuracy score :  0.9146805527738834
Accuracy score :  0.9146805527738834
params:
iterations: 1000
l2_leaf_reg: 3
depth: 3

Best_params:
Best_iterations: 1000
Best_l2_leaf_reg: 3
Best_depth: 2
Best_ac: 0.9111756459042659
Best_params:
Best_iterations: 1000
Best_l2_leaf_reg: 3
Best_depth: 3
Best_ac: 0.9146805527738834
Accuracy score :  0.9158822351291809
Accuracy score :  0.915882

## border

In [ ]:
from catboost import CatBoostClassifier

maxXG = 0
Best_numLeaves = 0
skipper = False
skipper = True

Best_iterations = 1000
Best_l2_leaf_reg = 3
Best_depth = 5
Best_border_count = 0

l2_leaf_regArr = [1,3,5,10,100]
border_countArr = [5,10,20,30,50,100,200]



for border_count in border_countArr:

  if(skipper):

    model = CatBoostClassifier(iterations=Best_iterations,border_count=border_count,l2_leaf_reg=Best_l2_leaf_reg,learning_rate=learnCAT_iHolder,depth=Best_depth)
    model.fit(X_train, Y_train,verbose=False)
    Y_pred = model.predict(X_test)
    ac = accuracy_score(Y_test, Y_pred)

    print("Accuracy score : ", ac)

    print("Accuracy score : ", ac)
    print("params:")
    print("iterations:",Best_iterations)
    print("l2_leaf_reg:",Best_l2_leaf_reg)
    print("border_count:",Best_border_count)
    print("depth:",Best_depth)
    


    print()
    print("Best_params:")
    print("Best_iterations:",Best_iterations)
    print("Best_l2_leaf_reg:",Best_l2_leaf_reg)
    print("Best_depth:",Best_depth)
    print("Best_border_count:",Best_border_count)
    print("Best_ac:",maxXG)


    if(ac > maxXG):
      maxXG = ac
      Best_border_count = border_count

      print("Best_params:")
      print("Best_iterations:",Best_iterations)
      print("Best_l2_leaf_reg:",Best_l2_leaf_reg)
      print("Best_depth:",Best_depth)
      print("Best_border_count:",Best_border_count)
      print("Best_ac:",maxXG)







print("Best_params:")
print("Best_iterations:",Best_iterations)
print("Best_l2_leaf_reg:",Best_l2_leaf_reg)
print("Best_depth:",Best_depth)
print("Best_border_count:",Best_border_count)
print("Best_ac:",maxXG)

Accuracy score :  0.9133787302223112
Accuracy score :  0.9133787302223112
params:
iterations: 1000
l2_leaf_reg: 3
border_count: 0
depth: 5

Best_params:
Best_iterations: 1000
Best_l2_leaf_reg: 3
Best_depth: 5
Best_border_count: 0
Best_ac: 0
Best_params:
Best_iterations: 1000
Best_l2_leaf_reg: 3
Best_depth: 5
Best_border_count: 5
Best_ac: 0.9133787302223112
Accuracy score :  0.9152813939515322
Accuracy score :  0.9152813939515322
params:
iterations: 1000
l2_leaf_reg: 3
border_count: 5
depth: 5

Best_params:
Best_iterations: 1000
Best_l2_leaf_reg: 3
Best_depth: 5
Best_border_count: 5
Best_ac: 0.9133787302223112
Best_params:
Best_iterations: 1000
Best_l2_leaf_reg: 3
Best_depth: 5
Best_border_count: 10
Best_ac: 0.9152813939515322
Accuracy score :  0.9145804125776087
Accuracy score :  0.9145804125776087
params:
iterations: 1000
l2_leaf_reg: 3
border_count: 10
depth: 5

Best_params:
Best_iterations: 1000
Best_l2_leaf_reg: 3
Best_depth: 5
Best_border_count: 10
Best_ac: 0.9152813939515322
Accu

In [ ]:
from catboost import CatBoostClassifier

maxXG = 0
Best_numLeaves = 0
skipper = False
skipper = True

Best_iterations = 1000
Best_l2_leaf_reg = 3
Best_depth = 5
# Best_border_count = 50
Best_min_data_in_leaf = 0


l2_leaf_regArr = [1,3,5,10,100]
border_countArr = [5,10,20,30,50,100,200]



for min_data_in_leaf in range(1,100):

  if(skipper):

    model = CatBoostClassifier(iterations=Best_iterations,min_data_in_leaf=min_data_in_leaf,border_count=Best_border_count,l2_leaf_reg=Best_l2_leaf_reg,learning_rate=learnCAT_iHolder,depth=Best_depth)
    model.fit(X_train, Y_train,verbose=False)
    Y_pred = model.predict(X_test)
    ac = accuracy_score(Y_test, Y_pred)

    print("Accuracy score : ", ac)

    print("Accuracy score : ", ac)
    print("params:")
    print("iterations:",Best_iterations)
    print("l2_leaf_reg:",Best_l2_leaf_reg)
    print("border_count:",Best_border_count)
    print("min_data_in_leaf:",min_data_in_leaf)    
    print("depth:",Best_depth)
    


    print()
    print("Best_params:")
    print("Best_iterations:",Best_iterations)
    print("Best_l2_leaf_reg:",Best_l2_leaf_reg)
    print("Best_depth:",Best_depth)
    print("Best_border_count:",Best_border_count)
    print("Best_min_data_in_leaf:",Best_min_data_in_leaf)
    
    print("Best_ac:",maxXG)


    if(ac > maxXG):
      maxXG = ac
      Best_min_data_in_leaf = min_data_in_leaf

      print("Best_params:")
      print("Best_iterations:",Best_iterations)
      print("Best_l2_leaf_reg:",Best_l2_leaf_reg)
      print("Best_depth:",Best_depth)
      print("Best_border_count:",Best_border_count)
      print("Best_min_data_in_leaf:",Best_min_data_in_leaf)
      print("Best_ac:",maxXG)







print("Best_params:")
print("Best_iterations:",Best_iterations)
print("Best_l2_leaf_reg:",Best_l2_leaf_reg)
print("Best_depth:",Best_depth)
print("Best_border_count:",Best_border_count)
print("Best_min_data_in_leaf:",Best_min_data_in_leaf)
print("Best_ac:",maxXG)

Accuracy score :  0.9182855998397756
Accuracy score :  0.9182855998397756
params:
iterations: 1000
l2_leaf_reg: 3
border_count: 30
min_data_in_leaf: 1
depth: 5

Best_params:
Best_iterations: 1000
Best_l2_leaf_reg: 3
Best_depth: 5
Best_border_count: 30
Best_min_data_in_leaf: 0
Best_ac: 0
Best_params:
Best_iterations: 1000
Best_l2_leaf_reg: 3
Best_depth: 5
Best_border_count: 30
Best_min_data_in_leaf: 1
Best_ac: 0.9182855998397756
Accuracy score :  0.9182855998397756
Accuracy score :  0.9182855998397756
params:
iterations: 1000
l2_leaf_reg: 3
border_count: 30
min_data_in_leaf: 2
depth: 5

Best_params:
Best_iterations: 1000
Best_l2_leaf_reg: 3
Best_depth: 5
Best_border_count: 30
Best_min_data_in_leaf: 1
Best_ac: 0.9182855998397756
Accuracy score :  0.9182855998397756
Accuracy score :  0.9182855998397756
params:
iterations: 1000
l2_leaf_reg: 3
border_count: 30
min_data_in_leaf: 3
depth: 5

Best_params:
Best_iterations: 1000
Best_l2_leaf_reg: 3
Best_depth: 5
Best_border_count: 30
Best_min_da